In [1]:
#load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datetime import datetime
from copy import deepcopy
import re, ast, os, difflib

In [2]:
current_file_path = os.path.dirname(os.path.abspath("__file__"))
data_path = os.path.join(current_file_path, "..", "data")
filename = "Carleton_Alberta_09_29_2023.csv"
file_path = os.path.join(data_path, filename)
raw_data = pd.read_csv(file_path)
HorV = "Home" # which side is your team at
op_HorV = "Home" if HorV == "Visitor" else "Visitor"

In [3]:
player_event_df = raw_data.copy()

In [4]:
player_event_df.iloc[2:8]

,Unnamed: 0,Time,Home,H-event,Score,V-event,Visitor
2,2,10:00,NaN,NaN,0 - 2,"Assist by ISHIMWE,LARS",Alberta
3,3,10:00,Carleton,"SIMPSON,NOAH made jump shot",2 - 2,NaN,NaN
4,4,10:00,NaN,NaN,2 - 2,"MEIKLEJOHN,BRANDON missed jump shot",Alberta
5,5,10:00,Carleton,"SPENCER,XAVIER defensive rebound",2 - 2,NaN,NaN
6,6,10:00,Carleton,"LATIFF,WAZIR missed jump shot",2 - 2,NaN,NaN
7,7,10:00,NaN,NaN,2 - 2,"POWELL,LOGAN defensive rebound",Alberta


In [5]:
pos_contrib = ['Assist', 'defensive rebound', 'made 3-pt. jump shot', 'made free throw',
                               'made jump shot', 'made layup', 'offensive rebound']
neg_contrib = ['Turnover', 'missed 3-pt. jump shot', 'missed free throw',
                               'missed jump shot', 'missed layup']

event_list = ['made layup','missed layup','Assist','Turnover','defensive rebound',
              'enters the game','goes to the bench','missed 3-pt. jump shot',
              'Foul','Steal','made free throw','missed free throw','made jump shot',
              'made 3-pt. jump shot','missed jump shot','offensive rebound']

In [6]:
def make_swap_uppernames(ls):
    formatted_players = []
    for player in ls:
        name_parts = player.split(" ")
        first_name = " ".join(name_parts[:-1])
        last_name = name_parts[-1]
        formatted_name = last_name.upper() + "," + first_name.upper()
        formatted_name = formatted_name.replace(".", "")
        formatted_players.append(formatted_name)

    return formatted_players


def players_list_and_starters(df:pd.DataFrame, quarter_index:int, HorV:str):
    p_dict = ast.literal_eval(df.iloc[quarter_index][HorV])
    p_list = p_dict["starters"].copy()
    p_list.extend(p_dict["reserves"])
    p_list.remove("Team")
    
    sts = p_dict["starters"].copy()

    p_list = make_swap_uppernames(p_list)
    sts = make_swap_uppernames(sts)

    return p_list, sts

### Simply building a dataframe from main one and getting kinds of event with respect to their player

In [7]:
pattern = "([A-Z]+\W*[A-Z]+,[A-Z]+\W*[A-Z]+)"
players_list, _ = players_list_and_starters(player_event_df, 0, HorV)
other_plist, _ = players_list_and_starters(player_event_df, 0, "Home" if HorV[0] == "V" else "Visitor")
player_event_df[f"H-event"] = player_event_df[f"H-event"].fillna("No Event")
player_event_df[f"V-event"] = player_event_df[f"V-event"].fillna("No Event")
for index, row in player_event_df.iterrows():
    for side in ["H", "V"]:
        compare_list = players_list.copy() if HorV[0] == side else other_plist.copy()
        player = re.search(pattern, row[f"{side}-event"])
        if player:
            player = player[0].strip()
            if player not in compare_list:
                found = difflib.get_close_matches(player, compare_list, n=1, cutoff=0.0)[0]
                if found:
                    player_event_df.loc[index, f"{side}_player"] = found
                else:
                    print(f"{player} not found !!!")
            else:
                player_event_df.loc[index, f"{side}_player"] = player
        else:
            player_event_df.loc[index, f"{side}_player"] = "No Player"
        
        for event in event_list:
            if event in row[f"{side}-event"]:
                player_event_df.loc[index, f"{side}_exactevent"] = event
                break
            else:
                player_event_df.loc[index, f"{side}_exactevent"] = "No Event"

In [8]:
player_event_df.head(n=20)

,Unnamed: 0,Time,Home,H-event,Score,V-event,Visitor,H_player,H_exactevent,V_player,V_exactevent
0,0,Quarter 1,"{'starters': ['Emmanuel Ugbah', 'Xavier Spencer', 'Wazir Latiff', 'Augustas Brazdeikis', 'Noah Horobetz Simpson'], 'reserves': ['Marjok Okado', 'Ben Riley', 'Aubrey Dorey-Havens', 'To Randriasalama', 'Raphael Constant', 'Reginald Jean Seraphin', 'Daniel Smith', 'Team']}",Quarter 1,Quarter 1,Quarter 1,"{'starters': ['Adam Paige', 'Lars Ishimwe', 'Isaac Simon', 'Brandon Meiklejohn', 'Patrick Fisher'], 'reserves': ['Logan Powell', 'Fahad Yusuf', 'Caiden Kushnir', 'Team']}",No Player,No Event,No Player,No Event
1,1,10:00,NaN,No Event,0 - 2,"MEIKLEJOHN,BRANDON made jump shot",Alberta,No Player,No Event,"MEIKLEJOHN,BRANDON",made jump shot
2,2,10:00,NaN,No Event,0 - 2,"Assist by ISHIMWE,LARS",Alberta,No Player,No Event,"ISHIMWE,LARS",Assist
3,3,10:00,Carleton,"SIMPSON,NOAH made jump shot",2 - 2,No Event,NaN,"SIMPSON,NOAH HOROBETZ",made jump shot,No Player,No Event
4,4,10:00,NaN,No Event,2 - 2,"MEIKLEJOHN,BRANDON missed jump shot",Alberta,No Player,No Event,"MEIKLEJOHN,BRANDON",missed jump shot
5,5,10:00,Carleton,"SPENCER,XAVIER defensive rebound",2 - 2,No Event,NaN,"SPENCER,XAVIER",defensive rebound,No Player,No Event
6,6,10:00,Carleton,"LATIFF,WAZIR missed jump shot",2 - 2,No Event,NaN,"LATIFF,WAZIR",missed jump shot,No Player,No Event
7,7,10:00,NaN,No Event,2 - 2,"POWELL,LOGAN defensive rebound",Alberta,No Player,No Event,"POWELL,LOGAN",defensive rebound
8,8,09:50,Carleton,"SPENCER,XAVIER made 3-pt. jump shot",5 - 2,No Event,NaN,"SPENCER,XAVIER",made 3-pt. jump shot,No Player,No Event
9,9,09:50,Carleton,"Assist by LATIFF,WAZIR",5 - 2,No Event,NaN,"LATIFF,WAZIR",Assist,No Player,No Event


### Calculating in-game time of each player on each quarter and events count for each player also

In [9]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# dependencies
players_list, _ = players_list_and_starters(player_event_df, 0, HorV)

player_event_df["Time"] = pd.to_datetime(player_event_df["Time"], format="%M:%S", errors="coerce")
under5min_df = player_event_df.loc[player_event_df["Time"] < datetime.strptime("05:00", "%M:%S")]

quarter_indices5min = list(reversed(under5min_df["Time"].nlargest(4).index))
quarter_indices = list(player_event_df[player_event_df['Score'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)

quarter_dict = {"player":players_list,
                "seconds1":[], "pts1":[], "ptc1":[],
                "seconds3":[], "pts3":[], "ptc3":[],
                "seconds2":[], "pts2":[], "ptc2":[],
                "seconds4":[], "pts4":[], "ptc4":[]}

v = list(quarter_dict.keys())
v.remove("player")
reorder_ls = ["lineup"] + v
lineup_quarter_dict = {key:[] for key in reorder_ls}

# Last 5 minutes statistics of quarters 2 and 4
quarter_dict5min = {"player":players_list, "time2":[],
                    "score2":[], "time4":[], "score4":[]}

quarter = 1
in_lineup = []

not_changed_list = ["not_changed" for n in range(len(players_list))]
event_num_dict = {k:[] for k in ["player"] + event_list + ["off_poss", "def_poss"]}
event_num_dict5min = {k:[] for k in ["player"] + [e + "2" for e in event_list] + [e + "4" for e in event_list]}
time_dict = {"player":players_list,
            "seconds":list(np.zeros(len(players_list))),
            "pts":list(np.zeros(len(players_list))),
            "ptc":list(np.zeros(len(players_list))),
            "timecache":not_changed_list.copy(),
            "ptscache":not_changed_list.copy(),
            "ptccache":not_changed_list.copy(),
            "seconds5min":list(np.zeros(len(players_list))),
            "points_conceded5min":list(np.zeros(len(players_list))),
            "timecache5min":not_changed_list.copy(),
            "scorecache5min":not_changed_list.copy(),}

# each custom minutes
custom_minute = 5
each_ls = list(reversed([int(m) for m in range(1, int((10 / custom_minute) + 1))]))
event_df_columns = []
eff_columns = [] # will be used for efficiency later on
for qu in range(1, 5):
    for each in list(reversed(each_ls)):
        eff_columns.append((f"quarter{qu}", f"{custom_minute}minute{each}"))
        for event in event_list:
            event_df_columns.append((f"quarter{qu}", f"{custom_minute}minute{each}", event))

event_df_columns = [("player", "player", "player")] + event_df_columns
init_data = {}
for c in event_df_columns:
    if "player" in c:
        init_data[c] = players_list
    else:
        init_data[c] = list(np.zeros(len(players_list)))

event_df_columns = pd.MultiIndex.from_tuples(event_df_columns)
event_df = pd.DataFrame(init_data, columns=event_df_columns)
minutes_ls = list(np.array(each_ls) * custom_minute) + [0]

lineup_time_dict = {k:[] for k in list(time_dict.keys())}
lineup_time_dict["lineup"] = lineup_time_dict.pop("player")
lineup_event_dict = {k:[] for k in ["lineup"] + event_list}

events_1min = pd.DataFrame(columns=[event_list])

for ind, row in player_event_df.iterrows():
    # 5min checking needs these constants
    cur_time = deepcopy(row["Time"])
    threshold_time = datetime.strptime("05:00", "%M:%S")
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
    # calculating in-game time of each player
    # -------------------------------------------
    # each quarter end calculations
    if ind in quarter_indices:

        if ind != len(player_event_df) - 1:
            _, starters = players_list_and_starters(player_event_df, ind, HorV)

        if ind != 0:
            for player in in_lineup:
                player_ind = time_dict["player"].index(player)
                cached_time = time_dict["timecache"][player_ind]
                ptscache = time_dict["ptscache"][player_ind]
                ptccache = time_dict["ptccache"][player_ind]
                if cached_time == "not_changed":
                    enter_time = datetime.strptime("10:00", "%M:%S")
                    enter_score_index = quarter_indices[quarter - 1] + 1
                    enter_pts = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "H" else 1])
                    enter_ptc = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "V" else 1])
                else:
                    enter_time = cached_time
                    enter_pts = ptscache
                    enter_ptc = ptccache
                
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()

                pts = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0 if HorV[0] == "H" else 1]) - enter_pts
                ptc = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0 if HorV[0] == "V" else 1]) - enter_ptc

                time_dict["seconds"][player_ind] += seconds
                time_dict["pts"][player_ind] += pts
                time_dict["ptc"][player_ind] += ptc
            
            # ---------------------
            # lineup quarter calculations
            lineup_cached_pts = lineup_time_dict["ptscache"][-1]
            lineup_cached_ptc = lineup_time_dict["ptccache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                lineup_time_dict["seconds"][-1] += 600
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_pts = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "H" else 1])
                enter_ptc = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "V" else 1])
            else:
                cached_time = lineup_time_dict["timecache"][-1]
                enter_time = cached_time
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()
                lineup_time_dict["seconds"][-1] += seconds
                enter_pts = int(lineup_cached_pts.split("-")[0 if HorV[0] == "H" else 1])
                enter_ptc = int(lineup_cached_pts.split("-")[0 if HorV[0] == "V" else 1])
            
            pts = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0 if HorV[0] == "H" else 1]) - enter_pts
            ptc = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0 if HorV[0] == "V" else 1]) - enter_ptc

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["pts"][-1] += pts
            lineup_time_dict["ptc"][-1] += ptc
            # ---------------------

            quarter_dict["player"] = time_dict["player"]
            quarter_dict[f"seconds{quarter}"] = time_dict["seconds"]
            quarter_dict[f"pts{quarter}"] = time_dict["pts"]
            quarter_dict[f"ptc{quarter}"] = time_dict["ptc"]
            if quarter in (2, 4):
                quarter_dict5min["player"] = time_dict["player"]
                quarter_dict5min[f"time{quarter}"] = time_dict["seconds5min"]
                quarter_dict5min[f"score{quarter}"] = time_dict["points_conceded5min"]

            for key in list(lineup_quarter_dict.keys()):
                if key != "lineup":
                    if int(key[-1]) == quarter:
                        if "seconds" in key:
                            lineup_quarter_dict[f"seconds{quarter}"].extend(lineup_time_dict["seconds"])
                        elif "pts" in key:
                            lineup_quarter_dict[f"pts{quarter}"].extend(lineup_time_dict["pts"])
                        else:
                            lineup_quarter_dict[f"ptc{quarter}"].extend(lineup_time_dict["ptc"])
                    else:
                        length = len(lineup_time_dict["lineup"])
                        zero_list = list(np.zeros(length))
                        lineup_quarter_dict[key].extend(zero_list)
                else:
                    lineup_quarter_dict[key].extend(lineup_time_dict["lineup"])

            quarter += 1
            time_dict["timecache"] = not_changed_list.copy()
            time_dict["ptscache"] = not_changed_list.copy()
            time_dict["ptccache"] = not_changed_list.copy()
            time_dict["timecache5min"] = not_changed_list.copy()
            time_dict["scorecache5min"] = not_changed_list.copy()
            time_dict["seconds"] = list(np.zeros(len(players_list)))
            time_dict["pts"] = list(np.zeros(len(players_list)))
            time_dict["ptc"] = list(np.zeros(len(players_list)))
            time_dict["seconds5min"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded5min"] = list(np.zeros(len(players_list)))

            lineup_time_dict = {key:[] for key in lineup_time_dict}

        in_lineup = starters.copy()
        lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
        for key in lineup_time_dict:
            if key != "lineup":
                if "cache" in key:
                    lineup_time_dict[key].append("not_changed")
                else:
                    lineup_time_dict[key].append(0)

        for key in list(lineup_event_dict.keys()):
            if key == "lineup":
                lineup_event_dict[key].append(sorted(in_lineup))
            else:
                lineup_event_dict[key].append(0)
        continue

    # counting off possession of opponent team which equals to def posession of selected team
    if row[f"{op_HorV[0]}_player"] not in ("No Player", np.nan, "nan") and row[f"{op_HorV[0]}_exactevent"] not in ("No Event", np.nan, "nan") and not pd.isna(row[f"{op_HorV[0]}_player"]) and not pd.isna(row[f"{op_HorV[0]}_exactevent"]):
        exactevent = row[f"{op_HorV[0]}_exactevent"]

        ## possession counting for players
        if exactevent in pos_contrib:
            for p in in_lineup:
                if p not in event_num_dict["player"]:
                    event_num_dict["player"].append(p)
                    for key in event_num_dict:
                        if key != "player":
                            event_num_dict[key].append(0)

                p_ind = event_num_dict["player"].index(p)
                event_num_dict["def_poss"][p_ind] += 1

    
    # -------------------------------------------
    # iterating rows calculation
    if row[f"{HorV[0]}_player"] not in ("No Player", np.nan, "nan") and row[f"{HorV[0]}_exactevent"] not in ("No Event", np.nan, "nan") and not pd.isna(row[f"{HorV[0]}_player"]) and not pd.isna(row[f"{HorV[0]}_exactevent"]):
        player_index = time_dict["player"].index(row[f"{HorV[0]}_player"])
        cached_time = time_dict["timecache"][player_index]
        ptscache = time_dict["ptscache"][player_index]
        ptccache = time_dict["ptccache"][player_index]
        if "goes to the bench" in row[f"{HorV[0]}_exactevent"]:
            try:
                in_lineup.remove(row[f"{HorV[0]}_player"])
            except:
                pass
            if cached_time == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_pts = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "H" else 1])
                enter_ptc = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "V" else 1])
            else:
                enter_time = cached_time
                enter_pts = ptscache
                enter_ptc = ptccache

            seconds = enter_time - row["Time"]
            seconds = seconds.total_seconds()

            pts = int(row["Score"].split("-")[0 if HorV[0] == "H" else 1]) - enter_pts
            ptc = int(row["Score"].split("-")[0 if HorV[0] == "V" else 1]) - enter_ptc


            time_dict["seconds"][player_index] += seconds
            time_dict["pts"][player_index] += pts
            time_dict["ptc"][player_index] += ptc

            ## if player goes to bench in last 5min of quarters 2 an 4 
            ## (considering not to exceed to to much memory and calculate them when needed)
            if cur_time < threshold_time and quarter in (2, 4):
                cached_time5min = time_dict["timecache5min"][player_index]
                cached_score5min = time_dict["scorecache5min"][player_index]
                if cached_time5min == "not_changed":
                    enter_time5min = threshold_time
                    # enter score should be reconsidered due to false index for 5min score and its alternation with first quarter row index 
                    enter_score_index5min = quarter_indices5min[quarter - 1] + 1
                    enter_score5min = int(player_event_df.iloc[enter_score_index5min]["Score"].split("-")[0 if HorV[0] == "V" else 1])
                else:
                    if cached_time5min > threshold_time:
                        enter_time5min = threshold_time
                        # enter score should be reconsidered due to false index for 5min score and its alternation with first quarter row index
                        enter_score_index5min = quarter_indices5min[quarter - 1] + 1
                        enter_score5min = int(player_event_df.iloc[enter_score_index5min]["Score"].split("-")[0 if HorV[0] == "V" else 1])
                    else:
                        enter_time5min = cached_time5min
                        enter_score5min = cached_score5min
                
                seconds5min = enter_time5min - row["Time"]
                seconds5min = seconds5min.total_seconds()
                points5min = int(row["Score"].split("-")[0 if HorV[0] == "V" else 1]) - enter_score5min
                time_dict["seconds5min"][player_index] += seconds5min
                time_dict["points_conceded5min"][player_index] += points5min

        elif "enters the game" in row[f"{HorV[0]}_exactevent"]:
            in_lineup.append(row[f"{HorV[0]}_player"])
            time_dict["timecache"][player_index] = row["Time"]
            time_dict["ptscache"][player_index] = int(row["Score"].split("-")[0 if HorV[0] == "H" else 1])
            time_dict["ptccache"][player_index] = int(row["Score"].split("-")[0 if HorV[0] == "V" else 1])
            
            ## time and score cache for under 5 min assessment would be catched if the the time is below 05:00 
            ## else i would not change it and it would be the initial list that i made before main loop for it
            if cur_time < threshold_time:
                time_dict["timecache5min"][player_index] = row["Time"]
                time_dict["scorecache5min"][player_index] = int(row["Score"].split("-")[0 if HorV[0] == "H" else 1])
        
        # ---------------------
        # lineup iterating rows calculation
        last_lineup = lineup_time_dict["lineup"][-1]
        if sorted(in_lineup) != sorted(last_lineup) and len(in_lineup) == 5:
            cached_time = lineup_time_dict["timecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_pts = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "H" else 1])
                enter_ptc = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0 if HorV[0] == "V" else 1])
            else:
                enter_time = cached_time
                enter_pts = int(lineup_time_dict["ptccache"][-1].split("-")[0 if HorV[0] == "H" else 1])
                enter_ptc = int(lineup_time_dict["ptccache"][-1].split("-")[0 if HorV[0] == "V" else 1])

            seconds = enter_time - row["Time"]
            seconds = seconds.total_seconds()

            pts_curr_score = int(row["Score"].split("-")[0 if HorV[0] == "H" else 1])
            ptc_curr_score = int(row["Score"].split("-")[0 if HorV[0] == "V" else 1])
            pts = pts_curr_score - enter_pts
            ptc = ptc_curr_score - enter_ptc

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["pts"][-1] += pts
            lineup_time_dict["ptc"][-1] += ptc

            lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
            for k in lineup_time_dict:
                if k != "lineup":
                    if "cache" in k:
                        lineup_time_dict[k].append("not_changed")
                    else:
                        lineup_time_dict[k].append(0)

            lineup_time_dict["timecache"].append(row["Time"])
            lineup_time_dict["ptscache"].append(row["Score"])
            lineup_time_dict["ptccache"].append(row["Score"])
        # ---------------------
    # -------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
        ## filling event_num_dict for meaesuring how many times each event occured
        exactevent = row[f"{HorV[0]}_exactevent"]
        if row[f"{HorV[0]}_player"] not in event_num_dict["player"]:
            event_num_dict["player"].append(row[f"{HorV[0]}_player"])
            for key in event_num_dict:
                if key != "player":
                    event_num_dict[key].append(0)
        
        player_index = event_num_dict["player"].index(row[f"{HorV[0]}_player"])
        event_num_dict[exactevent][player_index] += 1

        ## possession counting for players
        if exactevent in pos_contrib:
            for p in in_lineup:
                if p not in event_num_dict["player"]:
                    event_num_dict["player"].append(p)
                    for key in event_num_dict:
                        if key != "player":
                            event_num_dict[key].append(0)

                p_ind = event_num_dict["player"].index(p)
                event_num_dict["off_poss"][p_ind] += 1

        if cur_time < threshold_time and quarter in (2, 4):
            if row[f"{HorV[0]}_player"] not in event_num_dict5min["player"]:
                event_num_dict5min["player"].append(row[f"{HorV[0]}_player"])
                for key in event_num_dict5min:
                    if key != "player":
                        event_num_dict5min[key].append(0)
            
            player_index = event_num_dict5min["player"].index(row[f"{HorV[0]}_player"])
            event_num_dict5min[row[f"{HorV[0]}_exactevent"] + str(quarter)][player_index] += 1
        

        # each custom minutes
        minute_integer = row["Time"].minute + row["Time"].second / 60
        if minute_integer == 10:
            which_minute = 1
        else:
            for minute_index, minute in enumerate(minutes_ls):
                if minute <= minute_integer:
                    which_minute = minute_index
                    break
        
        event_df.loc[event_df["player", "player", "player"] == row[f"{HorV[0]}_player"], (f"quarter{quarter}", f"{custom_minute}minute{which_minute}", row[f"{HorV[0]}_exactevent"])] += 1

        # ---------------------
        # lineup event calculations
        if len(lineup_event_dict["lineup"]) == 0:
            lineup_event_dict["lineup"].append(sorted(in_lineup))
            for key in lineup_event_dict:
                if key != "lineup":
                    lineup_event_dict[key].append(0)

        last_lineup = lineup_event_dict["lineup"][-1]
        new_lineup = sorted(in_lineup.copy())
        # if row[f"{HorV[0]}_exactevent"] not in lineup_event_dict:
        #     lineup_event_dict[row[f"{HorV[0]}_exactevent"]] = list(np.zeros(len(lineup_event_dict["lineup"])))
        
        if new_lineup == last_lineup:
            lineup_event_dict[row[f"{HorV[0]}_exactevent"]][-1] += 1
        elif new_lineup != last_lineup and len(new_lineup) == 5:
            for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(new_lineup)
                else:
                    lineup_event_dict[key].append(0)
        # ---------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------

time_score_df = pd.DataFrame(quarter_dict)
time_score_df5min = pd.DataFrame(quarter_dict5min)
lineup_time_score_df = pd.DataFrame(lineup_quarter_dict)
event_num_df = pd.DataFrame(event_num_dict)
event_num_df5min = pd.DataFrame(event_num_dict5min)
lineup_event_df = pd.DataFrame(lineup_event_dict)

time_columns = []
time_columns_5min = []
for i in range(1, 5):
    for sub in ["seconds", "pts", "ptc"]:
        time_columns.append((f"quarter{i}", sub))
        if sub != "pts":
            time_columns_5min.append((f"quarter{i}", sub))

event_columns = []
for i in [2, 4]:
    for sub in event_list:
        event_columns.append((f"quarter{i}", sub))

time_score_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + time_columns)
time_score_df5min.columns = pd.MultiIndex.from_tuples([("player", "player")] + [(k, v) for k, v in time_columns_5min if k[-1] not in ("1", "3")])
lineup_time_score_df.columns = pd.MultiIndex.from_tuples([("lineup", "lineup")] + time_columns)
event_num_df5min.columns = pd.MultiIndex.from_tuples([("player", "player")] + event_columns)

time_sum_ls = [(f"quarter{i}", "seconds") for i in range(1, 5)]
pts_sum_ls = [(f"quarter{i}", "pts") for i in range(1, 5)]
ptc_sum_ls = [(f"quarter{i}", "ptc") for i in range(1, 5)]
time_score_df[("total", "seconds")] = time_score_df[time_sum_ls].sum(axis=1)
time_score_df[("total", "pts")] = time_score_df[pts_sum_ls].sum(axis=1)
time_score_df[("total", "ptc")] = time_score_df[ptc_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "seconds")] = lineup_time_score_df[time_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "pts")] = lineup_time_score_df[pts_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "ptc")] = lineup_time_score_df[ptc_sum_ls].sum(axis=1)

In [10]:
event_df

player   quarter1                               \
                    player   5minute1                                
                    player made layup missed layup Assist Turnover   
0           UGBAH,EMMANUEL        0.0          0.0    1.0      0.0   
1           SPENCER,XAVIER        0.0          0.0    0.0      0.0   
2             LATIFF,WAZIR        0.0          0.0    2.0      0.0   
3      BRAZDEIKIS,AUGUSTAS        0.0          0.0    0.0      0.0   
4    SIMPSON,NOAH HOROBETZ        0.0          0.0    0.0      0.0   
5             OKADO,MARJOK        0.0          0.0    0.0      0.0   
6                RILEY,BEN        0.0          0.0    0.0      0.0   
7      DOREY-HAVENS,AUBREY        0.0          0.0    0.0      1.0   
8         RANDRIASALAMA,TO        0.0          0.0    0.0      0.0   
9         CONSTANT,RAPHAEL        0.0          0.0    0.0      0.0   
10  SERAPHIN,REGINALD JEAN        0.0          0.0    0.0      0.0   
11            SMITH,DANIEL        0.0          0.0    0.0      0.0   

                                                                               \
                                                                                
   defensive rebound enters the game goes to the bench missed 3-pt. jump shot   
0                1.0             1.0               1.0                    0.0   
1                2.0             0.0               2.0                    0.0   
2                0.0             0.0               2.0                    1.0   
3                1.0             0.0               1.0                    1.0   
4                2.0             0.0               2.0                    0.0   
5                0.0             1.0               0.0                    0.0   
6                1.0             0.0               1.0                    0.0   
7                0.0             2.0               0.0                    0.0   
8                0.0             2.0               0.0                    0.0   
9                0.0             2.0               0.0                    0.0   
10               0.0             1.0               0.0                    0.0   
11               0.0             0.0               0.0                    0.0   

         ...          quarter4                                    \
         ...          5minute2                                     
   Foul  ... goes to the bench missed 3-pt. jump shot Foul Steal   
0   0.0  ...               1.0                    0.0  0.0   0.0   
1   1.0  ...               1.0                    0.0  2.0   1.0   
2   0.0  ...               0.0                    1.0  0.0   0.0   
3   0.0  ...               0.0                    0.0  0.0   0.0   
4   0.0  ...               2.0                    0.0  0.0   0.0   
5   0.0  ...               1.0                    0.0  0.0   0.0   
6   0.0  ...               1.0                    0.0  0.0   0.0   
7   0.0  ...               1.0                    0.0  1.0   0.0   
8   0.0  ...               1.0                    0.0  0.0   0.0   
9   0.0  ...               2.0                    0.0  0.0   0.0   
10  0.0  ...               1.0                    0.0  0.0   0.0   
11  0.0  ...               0.0                    0.0  0.0   0.0   

                                                                          \
                                                                           
   made free throw missed free throw made jump shot made 3-pt. jump shot   
0              0.0               0.0            0.0                  0.0   
1              0.0               0.0            0.0                  0.0   
2              0.0               0.0            0.0                  0.0   
3              0.0               0.0            1.0                  0.0   
4              2.0               0.0            0.0                  0.0   
5              0.0               0.0            0.0                  0.0   
6              0.0               0.0            

In [19]:
time_score_df

player quarter1             quarter2             quarter3  \
                    player  seconds   pts   ptc  seconds   pts   ptc  seconds   
0           UGBAH,EMMANUEL    749.0  22.0  34.0    600.0  13.0  16.0    338.0   
1           SPENCER,XAVIER    290.0  16.0  15.0    428.0   8.0  12.0    179.0   
2             LATIFF,WAZIR    451.0  21.0  26.0    354.0   5.0   6.0    104.0   
3      BRAZDEIKIS,AUGUSTAS    320.0  15.0  18.0    539.0  10.0  14.0    342.0   
4    SIMPSON,NOAH HOROBETZ    387.0  21.0  23.0    304.0   7.0  10.0    104.0   
5             OKADO,MARJOK    139.0   5.0   5.0      0.0   0.0   0.0    388.0   
6                RILEY,BEN    168.0   8.0  10.0     61.0   3.0   2.0      0.0   
7      DOREY-HAVENS,AUBREY    791.0  24.0  38.0    172.0   5.0   4.0    129.0   
8         RANDRIASALAMA,TO    533.0  14.0  19.0    246.0   8.0  10.0    108.0   
9         CONSTANT,RAPHAEL    597.0  14.0  22.0    230.0   3.0   4.0    366.0   
10  SERAPHIN,REGINALD JEAN    280.0   6.0  10.0     66.0   3.0   2.0    388.0   
11            SMITH,DANIEL     77.0   1.0   2.0      0.0   0.0   0.0    248.0   

              quarter4               total              
     pts  ptc  seconds   pts   ptc seconds   pts   ptc  
0    8.0  3.0    479.0  12.0   8.0  2166.0  55.0  61.0  
1    6.0  2.0    471.0  14.0  13.0  1368.0  44.0  42.0  
2    2.0  0.0    201.0   8.0   7.0  1110.0  36.0  39.0  
3   10.0  5.0    460.0  12.0  10.0  1661.0  47.0  47.0  
4    2.0  0.0    232.0   4.0   1.0  1027.0  34.0  34.0  
5   14.0  4.0    374.0   8.0   9.0   901.0  27.0  18.0  
6    0.0  0.0     53.0   0.0   1.0   282.0  11.0  13.0  
7    3.0  2.0    191.0   4.0   5.0  1283.0  36.0  49.0  
8    4.0  3.0    171.0   6.0   3.0  1058.0  32.0  35.0  
9   14.0  5.0    367.0  10.0  13.0  1560.0  41.0  44.0  
10  14.0  4.0      1.0   2.0   0.0   735.0  25.0  16.0  
11  12.0  4.0      0.0   0.0   0.0   325.0  13.0   6.0

In [20]:
time_score_df5min

player quarter2      quarter4      
                    player  seconds  ptc  seconds   ptc
0           UGBAH,EMMANUEL     38.0  0.0    101.0  16.0
1           SPENCER,XAVIER     75.0  3.0    209.0  17.0
2             LATIFF,WAZIR      0.0  0.0      0.0   0.0
3      BRAZDEIKIS,AUGUSTAS      0.0  0.0      0.0   0.0
4    SIMPSON,NOAH HOROBETZ      0.0  0.0    220.0   0.0
5             OKADO,MARJOK      0.0  0.0    189.0  -1.0
6                RILEY,BEN      0.0  0.0     43.0  -2.0
7      DOREY-HAVENS,AUBREY     60.0  0.0     60.0  -1.0
8         RANDRIASALAMA,TO      0.0  0.0    101.0  16.0
9         CONSTANT,RAPHAEL    170.0  2.0     69.0  14.0
10  SERAPHIN,REGINALD JEAN      0.0  0.0      1.0  -1.0
11            SMITH,DANIEL      0.0  0.0      0.0   0.0

In [11]:
event_num_df

,player,made layup,missed layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,Steal,made free throw,missed free throw,made jump shot,made 3-pt. jump shot,missed jump shot,offensive rebound,off_poss,def_poss
0,"UGBAH,EMMANUEL",0,0,1,2,2,3,4,0,1,1,1,0,3,0,4,1,120,122
1,"SPENCER,XAVIER",0,0,1,1,5,7,10,0,5,2,2,0,1,1,5,1,63,61
2,"LATIFF,WAZIR",0,0,3,0,0,3,5,2,1,0,0,0,0,1,3,0,45,46
3,"BRAZDEIKIS,AUGUSTAS",1,0,0,0,4,4,4,1,1,0,10,1,4,0,2,2,89,88
4,"SIMPSON,NOAH HOROBETZ",0,0,0,1,2,5,9,0,1,0,3,1,1,0,2,0,37,40
5,"RILEY,BEN",0,0,0,0,2,4,3,0,0,0,0,0,2,0,0,1,14,16
6,"CONSTANT,RAPHAEL",0,0,0,0,4,8,5,0,2,0,3,1,1,0,4,5,90,88
7,"RANDRIASALAMA,TO",0,0,0,1,3,5,4,1,1,0,0,0,1,1,1,0,74,66
8,"DOREY-HAVENS,AUBREY",0,0,1,2,1,9,7,0,2,0,2,2,1,1,1,1,87,89
9,"OKADO,MARJOK",0,0,1,0,1,5,4,1,0,1,0,0,1,1,2,0,49,39


In [22]:
event_num_df5min

player   quarter2                               \
                    player made layup missed layup Assist Turnover   
0      DOREY-HAVENS,AUBREY          0            0      0        0   
1           SPENCER,XAVIER          0            0      0        1   
2           UGBAH,EMMANUEL          0            0      0        0   
3             SMITH,DANIEL          0            0      1        0   
4         CONSTANT,RAPHAEL          0            0      0        0   
5      BRAZDEIKIS,AUGUSTAS          0            0      0        0   
6             OKADO,MARJOK          0            0      0        0   
7   SERAPHIN,REGINALD JEAN          0            0      0        1   
8         RANDRIASALAMA,TO          0            0      0        0   
9    SIMPSON,NOAH HOROBETZ          0            0      0        0   
10               RILEY,BEN          0            0      0        0   
11            LATIFF,WAZIR          0            0      0        0   

                                                                               \
   defensive rebound enters the game goes to the bench missed 3-pt. jump shot   
0                  0               2                 2                      0   
1                  0               3                 3                      0   
2                  0               0                 1                      0   
3                  1               1                 0                      0   
4                  2               0                 1                      0   
5                  1               1                 0                      0   
6                  0               0                 0                      0   
7                  0               0                 0                      0   
8                  0               0                 0                      0   
9                  0               0                 0                      0   
10                 0               0                 0                      0   
11                 0               0                 0                      0   

         ...          quarter4                                    \
   Foul  ... goes to the bench missed 3-pt. jump shot Foul Steal   
0     0  ...                 1                      0    1     0   
1     1  ...                 1                      0    2     1   
2     0  ...                 1                      0    0     0   
3     1  ...                 0                      0    0     0   
4     0  ...                 2                      0    0     0   
5     0  ...                 0                      0    0     0   
6     0  ...                 1                      0    0     0   
7     0  ...                 1                      0    0     0   
8     0  ...                 1                      0    0     0   
9     0  ...                 2                      0    0     0   
10    0  ...                 1                      0    0     0   
11    0  ...                 0                      1    0     0   

                                                                          \
   made free throw missed free throw made jump shot made 3-pt. jump shot   
0                0                 0              0                    0   
1                0                 0              0                    0   
2                0                 0              0                    0   
3                0                 0              0                    0   
4                0                 0              1                    0   
5                0                 0              1                    0   
6                0                 0              0                    0   
7                2                 0              0                    0   
8                0                 0              0                    0   
9                2                 0              0                    0   
10               0                 0   

In [23]:
lineup_event_df # starter lineup will be added automatically at end of the match so we ignore it cause no event will happen there(try proving it by removing .iloc)

,lineup,made layup,missed layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,Steal,made free throw,missed free throw,made jump shot,made 3-pt. jump shot,missed jump shot,offensive rebound
0,"[BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]",0,0,1,0,3,0,0,0,0,0,1,1,1,1,4,0
1,"[BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]",0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,"[BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL]",0,0,0,1,2,0,0,0,0,0,0,0,2,0,2,2
3,"[CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL]",0,0,0,0,2,0,0,0,0,0,2,2,0,0,0,0
4,"[CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]",0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
5,"[CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL]",0,0,1,0,3,0,1,0,0,0,2,0,0,2,0,0
6,"[BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL, SPENCER,XAVIER]",0,0,0,1,1,0,0,0,0,0,2,0,1,0,1,1
7,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL]",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,"[BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]",0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0
9,"[BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL]",1,0,0,0,1,0,0,0,1,0,0,1,0,0,2,0


In [24]:
import pandas as pd

# Assuming you have a dataframe called "lineup_event_df" with columns "lineup", "Assist", and "Turnover"

# Convert the "lineup" column to a tuple
lineup_event_df["lineup"] = lineup_event_df["lineup"].apply(tuple)

# # Convert the "Assist" and "Turnover" columns to numeric types
# lineup_event_df["Assist"] = pd.to_numeric(lineup_event_df["Assist"], errors="coerce")
# lineup_event_df["Turnover"] = pd.to_numeric(lineup_event_df["Turnover"], errors="coerce")

# Group the dataframe by "lineup" column (now converted to a tuple) and calculate the sum of "Assist" and "Turnover"
cols = lineup_event_df.columns.to_list().copy()
cols.remove("lineup")
lineup_event_df = lineup_event_df.groupby("lineup").sum().reset_index()

# Print the resulting grouped dataframe
lineup_event_df

,lineup,made layup,missed layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,Steal,made free throw,missed free throw,made jump shot,made 3-pt. jump shot,missed jump shot,offensive rebound
0,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL)",1,0,0,2,4,0,0,0,2,0,0,1,2,0,4,2
1,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, OKADO,MARJOK, SPENCER,XAVIER)",0,0,0,0,0,0,0,0,0,0,2,0,1,0,2,1
2,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, RILEY,BEN, UGBAH,EMMANUEL)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, SPENCER,XAVIER, UGBAH,EMMANUEL)",0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0
4,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER)",0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
5,"(BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, RANDRIASALAMA,TO, SPENCER,XAVIER, UGBAH,EMMANUEL)",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1
6,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL)",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL)",0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
8,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL)",0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
9,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL)",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [25]:
lineup_time_score_df

lineup  \
                                                                                             lineup   
0        [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
1        [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
2    [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL]   
3     [CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL]   
4     [CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]   
5       [CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL]   
6         [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL, SPENCER,XAVIER]   
7    [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL]   
8        [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
9    [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL]   
10        [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, RANDRIASALAMA,TO, SPENCER,XAVIER, UGBAH,EMMANUEL]   
11   [BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
12       [DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, RILEY,BEN, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL]   
13      [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER, UGBAH,EMMANUEL]   
14       [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
15            [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, SPENCER,XAVIER, UGBAH,EMMANUEL]   
16       [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
17              [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, OKADO,MARJOK, SPENCER,XAVIER]   
18                   [CONSTANT,RAPHAEL, LATIFF,WAZIR, OKADO,MARJOK, SPENCER,XAVIER, UGBAH,EMMANUEL]   
19            [CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SPENCER,XAVIER, UGBAH,EMMANUEL]   
20            [CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SPENCER,XAVIER, UGBAH,EMMANUEL]   
21   [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL]   
22        [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, RANDRIASALAMA,TO, SPENCER,XAVIER, UGBAH,EMMANUEL]   
23   [BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
24                 [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER]   
25       [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL]   
26  [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL]   
27         [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL]   
28        [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL]   
29                 [BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, RILEY,BEN, UGBAH,EMMANUEL]   

   quarter1             quarter2           quarter3           quarter4       \
    seconds   pts   ptc  seconds  pts  ptc  seconds  pts  ptc  seconds  pts   
0     637.0  21.0  28.0      0.0  0.0  0.0      0.0  0.0  0.0      0.0  0.0   
1       0.0   0.0   0.0      0.0  0.0  0.0    104.0  2.0  0.0      0.0  0.0   
2       0.0   0.0   0.0      0.0  0.0  0.0    108.0  4.0  3.0      0.0  0.0   
3       0.0   0.0   0.0      0.0  0.0  0.0    126.0  2.0  0.0      0.0  0.0   
4       0.0   0.0   0.0      0.0  0.0  0.0     14.0  0.0  0.0      0.0  0.0   
5       0.0   0.0   0.0      0.0  0.0  0.0    118.0  8.0  2.0      0.0  0.0   
6       0.0   0.0   0.0      0.0  0.0  0.0     61.0  4.0  2.0      0.0  0.0   
7       0.0   0.0   0.0      0.0  0.0  0.0    138.0  0.0  0.0      0.0  0.0   
8     

In [26]:
lineup_time_score_df["lineup", "lineup"] = lineup_time_score_df["lineup", "lineup"].apply(tuple)
lineup_time_score_df = lineup_time_score_df.groupby(("lineup", "lineup")).sum().reset_index()
lineup_time_score_df

lineup  \
                                                                                             lineup   
0    (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, UGBAH,EMMANUEL)   
1               (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, OKADO,MARJOK, SPENCER,XAVIER)   
2                  (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, RILEY,BEN, UGBAH,EMMANUEL)   
3             (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, LATIFF,WAZIR, SPENCER,XAVIER, UGBAH,EMMANUEL)   
4                  (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, OKADO,MARJOK, RILEY,BEN, SPENCER,XAVIER)   
5         (BRAZDEIKIS,AUGUSTAS, CONSTANT,RAPHAEL, RANDRIASALAMA,TO, SPENCER,XAVIER, UGBAH,EMMANUEL)   
6    (BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL)   
7   (BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL)   
8         (BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL)   
9          (BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, UGBAH,EMMANUEL)   
10      (BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER, UGBAH,EMMANUEL)   
11       (BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL)   
12        (BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL, SPENCER,XAVIER)   
13       (BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL)   
14   (BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL)   
15            (CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SPENCER,XAVIER, UGBAH,EMMANUEL)   
16      (CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SMITH,DANIEL)   
17    (CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER)   
18    (CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL)   
19            (CONSTANT,RAPHAEL, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SPENCER,XAVIER, UGBAH,EMMANUEL)   
20                   (CONSTANT,RAPHAEL, LATIFF,WAZIR, OKADO,MARJOK, SPENCER,XAVIER, UGBAH,EMMANUEL)   
21       (DOREY-HAVENS,AUBREY, RANDRIASALAMA,TO, RILEY,BEN, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL)   

   quarter1             quarter2           quarter3           quarter4       \
    seconds   pts   ptc  seconds  pts  ptc  seconds  pts  ptc  seconds  pts   
0       0.0   0.0   0.0    111.0  2.0  2.0    108.0  4.0  3.0     38.0  0.0   
1       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     78.0  4.0   
2       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     20.0  0.0   
3       0.0   0.0   0.0    194.0  1.0  2.0      0.0  0.0  0.0      0.0  0.0   
4       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     43.0  0.0   
5       0.0   0.0   0.0     22.0  0.0  0.0      0.0  0.0  0.0     58.0  2.0   
6       0.0   0.0   0.0      0.0  0.0  0.0    138.0  0.0  0.0      0.0  0.0   
7       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     60.0  0.0   
8       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0      1.0  2.0   
9       0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     20.0  0.0   
10      0.0   0.0   0.0      5.0  0.0  0.0      0.0  0.0  0.0      0.0  0.0   
11    637.0  21.0  28.0    252.0  4.0  4.0    104.0  2.0  0.0     12.0  0.0   
12      0.0   0.0   0.0      0.0  0.0  0.0     61.0  4.0  2.0      0.0  0.0   
13      0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     65.0  0.0   
14      0.0   0.0   0.0     52.0  3.0  6.0      0.0  0.0  0.0     75.0  4.0   
15      0.0   0.0   0.0      0.0  0.0  0.0      0.0  0.0  0.0     33.0  2.0   
16      0.0   0.0   0.0      0.0  0.0  0.0    118.0  8.0  2.0      0.0  0.0   
17      0.0   0.0   0.0      0.0  0.0  0.0     14.0  0.0  0.0      0.0  0.0   
18      0.0   0.0   0.0      0.0  0.0  0

off possession = made and miss

def possession = Turnover, defensive rebound,	Foul, Steal

efficiency = point, rebound, assist, steal, block, -missed free throw, -turnover, -points conceded

What type of game was it? (Pre-season, Regular Season, or Playoffs)

In [27]:
def cal_eff(offense, defense, time):
    eff = ((offense - defense) * 60) / time
    return float(eff)

In [28]:
pos_contrib = ['Assist', 'defensive rebound', 'made 3-pt. jump shot', 'made free throw',
                               'made jump shot', 'made layup', 'offensive rebound']
neg_contrib = ['Turnover', 'missed 3-pt. jump shot', 'missed free throw',
                               'missed jump shot', 'missed layup']

eff_df = pd.DataFrame({key:[] for key in [("player", "player")] + eff_columns})
eff_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + eff_columns)
for index, row in event_df.iterrows():
    data = {("player", "player"):[row["player", "player", "player"]]}
    for col in eff_columns:
        alter = []
        for event in pos_contrib:
            alter.append(col + (event,))

        offense = row[alter].sum()

        for event in neg_contrib:
            alter.append(col + (event,))

        defense = row[alter].sum()

        eff = cal_eff(offense, defense, custom_minute * 60)
        data[col] = [eff]
    
    new_df = pd.DataFrame(data)
    eff_df = pd.concat([eff_df, new_df], ignore_index=True, axis=0)

In [29]:
eff_df

player quarter1          quarter2          quarter3  \
                    player 5minute1 5minute2 5minute1 5minute2 5minute1   
0           UGBAH,EMMANUEL      0.0      0.0     -0.4      0.0     -0.4   
1           SPENCER,XAVIER     -0.2     -0.2      0.0     -0.2      0.0   
2             LATIFF,WAZIR     -0.6      0.0      0.0      0.0      0.0   
3      BRAZDEIKIS,AUGUSTAS     -0.2      0.0     -0.2      0.0     -0.2   
4    SIMPSON,NOAH HOROBETZ     -0.6      0.0      0.0      0.0     -0.2   
5             OKADO,MARJOK     -0.2     -0.2      0.0      0.0      0.0   
6                RILEY,BEN      0.0      0.0      0.0      0.0      0.0   
7      DOREY-HAVENS,AUBREY     -0.2      0.0      0.0     -0.4     -0.2   
8         RANDRIASALAMA,TO      0.0      0.0      0.0      0.0     -0.4   
9         CONSTANT,RAPHAEL      0.0     -0.4      0.0      0.0     -0.2   
10  SERAPHIN,REGINALD JEAN      0.0      0.0      0.0     -0.4     -0.2   
11            SMITH,DANIEL      0.0      0.0      0.0      0.0      0.0   

            quarter4           
   5minute2 5minute1 5minute2  
0      -0.2      0.0     -0.2  
1      -0.2     -0.4      0.0  
2      -0.2      0.0     -0.2  
3      -0.2      0.0      0.0  
4       0.0      0.0      0.0  
5       0.0      0.0     -0.2  
6       0.0      0.0      0.0  
7       0.0     -0.2      0.0  
8       0.0      0.0     -0.2  
9       0.0     -0.4      0.0  
10      0.0      0.0      0.0  
11      0.0      0.0      0.0

In [30]:
scoring_values = {'made layup': 2, 'made free throw': 1, 'made jump shot': 2, 'made 3-pt. jump shot': 3}

neg_contrib2 = []
neg_contrib4 = []
pos_contrib2 = []
pos_contrib4 = []
for q in [2, 4]:
    for n in neg_contrib:
        globals()[f"neg_contrib{q}"].append((f"quarter{q}", n))
    for p in pos_contrib:
        globals()[f"pos_contrib{q}"].append((f"quarter{q}", n))

final_columns = ['Player Name', 'PtScored', 'ptsconceded', "OffRtg", "DefRtg", "NetRtg",
                'total off possession', 'total def possession', "global efficiency",
                "quarter2 last 5min efficiency", "quarter4 last 5min efficiency",
                'minutes', 'home/visitor', 'opponent', 'date', 'game_type']

player_final_table = pd.DataFrame(columns=final_columns)
for index, row in event_num_df.iterrows():
    
    points_scored = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "pts")].to_list()[0])
    points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "ptc")].to_list()[0])

    seconds = time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "seconds")]
    time = seconds.iloc[0]
    global_off_possession = row["off_poss"]
    global_def_possession = row["def_poss"]
    global_efficiency = cal_eff(global_off_possession, global_def_possession, time)

    if row["player"] in event_num_df5min["player", "player"].tolist():
        time_row5min = time_score_df5min.loc[time_score_df5min["player", "player"] == row["player"]]
        event_row5min = event_num_df5min.loc[event_num_df5min["player", "player"] == row["player"]]
        for q in [2, 4]:
            time = time_row5min[f"quarter{q}", "seconds"].iloc[0]
            offense = float(event_row5min[globals()[f"pos_contrib{q}"]].sum(axis=1).iloc[0])
            defense = float(event_row5min[globals()[f"neg_contrib{q}"]].sum(axis=1).iloc[0])
            globals()[f"quarter{q}_5min_eff"] = cal_eff(offense, defense, time)

            if time == 0:
                globals()[f"quarter{q}_5min_eff"] = "Not in the time"
    else:
        quarter2_5min_eff = "Not in the time"
        quarter4_5min_eff = "Not in the time"
    
    minutes = seconds / 60
    minutes = "{:.2f}".format(minutes.to_list()[0])
    hv_df = player_event_df.loc[player_event_df[f"{HorV[0]}_player"] == row["player"]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"
    
    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    try:
        off_rtg = (100 * points_scored) / (global_off_possession + global_def_possession)
        def_rtg = (100 * points_conceded) / (global_off_possession + global_def_possession)
    except ZeroDivisionError:
        off_rtg = 0
        def_rtg = 0        

    net_rtg = off_rtg - def_rtg
    off_rtg = "{:.3f}".format(off_rtg)
    def_rtg = "{:.3f}".format(def_rtg)
    net_rtg = "{:.3f}".format(net_rtg)

    new_row = {"Player Name":[row["player"]], "PtScored":[points_scored], "OffRtg":[off_rtg], "DefRtg":[def_rtg],
               "NetRtg":[net_rtg], "ptsconceded":[points_conceded], 'total off possession':[global_off_possession], 
               'total def possession':[global_def_possession], "global efficiency":[global_efficiency], 
               "quarter2 last 5min efficiency":[quarter2_5min_eff], "quarter4 last 5min efficiency":[quarter4_5min_eff],
               "minutes":minutes, "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    player_final_table = pd.concat([player_final_table, new_df], ignore_index=True, axis=0)

player_final_table = player_final_table.reindex(columns=final_columns)

/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeWarning: invalid value encountered in scalar divide
  eff = ((offense - defense) * 60) / time
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeWarning: invalid value encountered in scalar divide
  eff = ((offense - defense) * 60) / time
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  eff = ((offense - defense) * 60) / time
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeWarning: invalid value encountered in scalar divide
  eff = ((offense - defense) * 60) / time
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeWarning: invalid value encountered in scalar divide
  eff = ((offense - defense) * 60) / time
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2880/1392762153.py:2: RuntimeW

## pts is team scoring while the player is inside !!!

In [31]:
player_final_table

,Player Name,PtScored,ptsconceded,OffRtg,DefRtg,NetRtg,total off possession,total def possession,global efficiency,quarter2 last 5min efficiency,quarter4 last 5min efficiency,minutes,home/visitor,opponent,date,game_type
0,"SIMPSON,NOAH HOROBETZ",34.0,34.0,91.892,91.892,0.000,26,11,0.876339,Not in the time,0.0,17.12,Home,Alberta,NaN,NaN
1,"UGBAH,EMMANUEL",55.0,61.0,45.833,50.833,-5.000,79,41,1.052632,0.0,-0.594059,36.10,Home,Alberta,NaN,NaN
2,"SPENCER,XAVIER",44.0,42.0,69.841,66.667,3.175,43,20,1.008772,-0.8,0.0,22.80,Home,Alberta,NaN,NaN
3,"LATIFF,WAZIR",36.0,39.0,80.000,86.667,-6.667,29,16,0.702703,Not in the time,Not in the time,18.50,Home,Alberta,NaN,NaN
4,"BRAZDEIKIS,AUGUSTAS",47.0,47.0,52.809,52.809,0.000,59,30,1.047562,Not in the time,Not in the time,27.68,Home,Alberta,NaN,NaN
5,"RILEY,BEN",11.0,13.0,78.571,92.857,-14.286,8,6,0.425532,Not in the time,0.0,4.70,Home,Alberta,NaN,NaN
6,"CONSTANT,RAPHAEL",41.0,44.0,45.556,48.889,-3.333,58,32,1.000000,0.0,0.0,26.00,Home,Alberta,NaN,NaN
7,"RANDRIASALAMA,TO",32.0,35.0,43.243,47.297,-4.054,46,28,1.020794,Not in the time,-0.594059,17.63,Home,Alberta,NaN,NaN
8,"DOREY-HAVENS,AUBREY",36.0,49.0,41.379,56.322,-14.943,57,30,1.262666,-2.0,0.0,21.38,Home,Alberta,NaN,NaN
9,"OKADO,MARJOK",27.0,18.0,55.102,36.735,18.367,35,14,1.398446,Not in the time,-0.31746,15.02,Home,Alberta,NaN,NaN


## Let's go to lineup

In [24]:
lineup_event_df

,lineup,made layup,missed layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,Steal,made free throw,missed free throw,made jump shot,made 3-pt. jump shot,missed jump shot,offensive rebound
0,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL)",0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0
1,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER)",0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL)",0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
3,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL)",0,0,0,2,0,0,0,0,1,0,0,0,0,0,1,0
4,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER)",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL, SPENCER,XAVIER)",0,0,2,1,4,0,0,2,3,0,5,0,2,1,7,7
6,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL)",0,0,3,5,8,0,0,4,4,0,0,3,3,2,0,0
7,"(BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL, SPENCER,XAVIER)",0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0
8,"(BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL, SPENCER,XAVIER)",0,0,2,5,3,0,0,3,1,0,1,1,1,1,1,1
9,"(DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, SERAPHIN,REGINALD JEAN, SMITH,DANIEL)",0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0


In [30]:
if "Player Name" in final_columns:
    final_columns.remove("Player Name")

lineup_final_columns = ["Lineup"].extend(final_columns)
lineup_final_table = pd.DataFrame(columns=[lineup_final_columns])
for index, row in lineup_event_df.iterrows():
    
    points_scored = float(lineup_time_score_df.loc[lineup_time_score_df["lineup", "lineup"] == tuple(row["lineup"])][("total", "pts")].to_list()[0])
    points_conceded = float(lineup_time_score_df.loc[lineup_time_score_df[("lineup", "lineup")] == tuple(row["lineup"])][("total", "ptc")].to_list()[0])

    seconds = lineup_time_score_df.loc[lineup_time_score_df[("lineup", "lineup")] == row["lineup"]][("total", "seconds")]
    time = seconds.iloc[0]
    global_off_possession = row[pos_contrib].sum()
    global_def_possession = row[neg_contrib].sum()
    global_efficiency = cal_eff(global_off_possession, global_def_possession, time)

    try:
        off_rtg = (100 * points_scored) / (global_off_possession + global_def_possession)
        def_rtg = (100 * points_conceded) / (global_off_possession + global_def_possession)
    except ZeroDivisionError:
        off_rtg = 0
        def_rtg = 0        

    net_rtg = off_rtg - def_rtg
    off_rtg = "{:.3f}".format(off_rtg)
    def_rtg = "{:.3f}".format(def_rtg)
    net_rtg = "{:.3f}".format(net_rtg)

    minutes = lineup_time_score_df["total", "seconds"].iloc[index] / 60
    minutes = "{:.2f}".format(minutes)

    hv_df = player_event_df.loc[player_event_df[f"{HorV[0]}_player"] == row["lineup"][0]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"

    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Lineup":[row["lineup"]], "PtScored":[points_scored],
               "OffRtg":off_rtg, "DefRtg":def_rtg, "NetRtg":net_rtg,
               "ptsconceded":[points_conceded], "global off possession":[global_off_possession], 
               "global def possession":global_def_possession, "efficiency":[global_efficiency], 
               "minutes":minutes, "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    lineup_final_table = pd.concat([lineup_final_table, new_df], ignore_index=True, axis=0)

lineup_final_table = lineup_final_table.reindex(columns=lineup_final_columns)

In [28]:
eff_columns

[('quarter1', '5minute1'),
 ('quarter1', '5minute2'),
 ('quarter2', '5minute1'),
 ('quarter2', '5minute2'),
 ('quarter3', '5minute1'),
 ('quarter3', '5minute2'),
 ('quarter4', '5minute1'),
 ('quarter4', '5minute2')]

In [26]:
lineup_final_table

,None,Lineup,PtScored,OffRtg,DefRtg,NetRtg,ptsconceded,global off possession,global def possession,efficiency,minutes,home/visitor,opponent
0,NaN,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL)",3.0,75.000,75.000,0.000,3.0,3.0,1.0,1.935484,1.03,Visitor,Ottawa
1,NaN,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER)",2.0,100.000,0.000,100.000,0.0,1.0,1.0,0.000000,0.05,Visitor,Ottawa
2,NaN,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL)",3.0,75.000,0.000,75.000,0.0,3.0,1.0,2.553191,0.78,Visitor,Ottawa
3,NaN,"(BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL)",0.0,0.000,166.667,-166.667,5.0,0.0,3.0,-2.769231,1.08,Visitor,Ottawa
4,NaN,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER)",0.0,0.000,0.000,0.000,0.0,0.0,1.0,-3.529412,0.28,Visitor,Ottawa
5,NaN,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL, SPENCER,XAVIER)",12.0,38.710,22.581,16.129,7.0,21.0,10.0,2.200000,5.00,Visitor,Ottawa
6,NaN,"(BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH HOROBETZ, SPENCER,XAVIER, UGBAH,EMMANUEL)",9.0,32.143,67.857,-35.714,19.0,16.0,12.0,0.489796,8.17,Visitor,Ottawa
7,NaN,"(BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL, SPENCER,XAVIER)",3.0,100.000,33.333,66.667,1.0,2.0,1.0,2.307692,0.43,Visitor,Ottawa
8,NaN,"(BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH HOROBETZ, SMITH,DANIEL, SPENCER,XAVIER)",6.0,31.579,57.895,-26.316,11.0,9.0,10.0,-0.215827,4.63,Visitor,Ottawa
9,NaN,"(DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, SERAPHIN,REGINALD JEAN, SMITH,DANIEL)",2.0,66.667,0.000,66.667,0.0,2.0,1.0,1.224490,0.82,Visitor,Ottawa


## New Idea: maybe some AI for new-lineup would be good to estimate its statistics were !!!